<a href="https://colab.research.google.com/github/Grashch/Data-science/blob/main/TreeRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [104]:
data = pd.read_csv('/content/flavors_of_cacao.csv')
data = data.rename(columns={
    'Company\xa0\n(Maker-if known)': 'Company (Maker-if known)',
    'Specific Bean Origin\nor Bar Name': 'Specific Bean Origin or Bar Name',
    'Review\nDate' : 'Review Date',
    'Cocoa\nPercent' : 'Cocoa Percent',
    'Company\nLocation' : 'Company Location',
    'Bean\nType' : 'Bean Type',
    'Broad Bean\nOrigin' : 'Broad Bean Origin'
})
data['Cocoa Percent'] = data['Cocoa Percent'].str.replace('%', '').astype(float)
data.head()

,Company (Maker-if known),Specific Bean Origin or Bar Name,REF,Review Date,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70.0,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,,Peru


In [105]:
data.isna().mean(axis=0)

,0
Company (Maker-if known),0.000000
Specific Bean Origin or Bar Name,0.000000
REF,0.000000
Review Date,0.000000
Cocoa Percent,0.000000
Company Location,0.000000
Rating,0.000000
Bean Type,0.000557
Broad Bean Origin,0.000557


In [106]:
categorical_features = []
numerical_features = []
[col for col in data.columns if data[col].dtype == "object"]

['Company (Maker-if known)',
 'Specific Bean Origin or Bar Name',
 'Company Location',
 'Bean Type',
 'Broad Bean Origin']

In [107]:
company_stat = data['Company (Maker-if known)'].value_counts()
company_stat

,count
Company (Maker-if known),
Soma,47
Bonnat,27
Fresco,26
Pralus,25
A. Morin,23
...,...
Svenska Kakaobolaget,1
Baravelli's,1
Urzi,1


In [108]:
data['company'] = data['Company (Maker-if known)'].apply(lambda x: x if company_stat[x] >= 18 else 'Other')
data['company'].value_counts()

,count
company,
Other,1505
Soma,47
Bonnat,27
Fresco,26
Pralus,25
A. Morin,23
Arete,22
Guittard,22
Domori,22


In [109]:
origin_stat = data['Specific Bean Origin or Bar Name'].value_counts()
location_stat = data['Company Location'].value_counts()
type_stat = data['Bean Type'].value_counts(dropna=False)
bean_stat = data['Broad Bean Origin'].value_counts(dropna=False)

In [110]:
data['origin'] = data['Specific Bean Origin or Bar Name'].apply(lambda x: x if origin_stat[x] >= 18 else 'Other')
data['location'] = data['Company Location'].apply(lambda x: x if location_stat[x] >= 18 else 'Other')
data['bean_type'] = data['Bean Type'].apply(lambda x: x if type_stat[x] >= 20 else 'Other')
data['bean_origin'] = data['Broad Bean Origin'].apply(lambda x: x if bean_stat[x] >= 18 else 'Other')

In [111]:
categorical_features.extend(['company', 'origin', 'location', 'bean_type', 'bean_origin'])
numerical_features.extend(['REF', 'Review Date', 'Cocoa Percent'])

In [112]:
sum([data[feature].nunique() for feature in categorical_features])

70